In [1]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from lib.sampling import subsampling
import numpy as np
from sklearn.model_selection import GridSearchCV
from lib.mutation_util import date
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from lib.sampling import subsampling
import numpy as np
from sklearn.model_selection import GridSearchCV
from lib.mutation_util import date
from sklearn import metrics
%load_ext autoreload
%autoreload

In [4]:
test_start_date = "2021-12-01"
test_end_date = "2021-12-31"

In [6]:
df_ori = pd.read_csv(f"../adyen-dataset.csv")
df_ori["date"] = df_ori.apply(date, axis=1)
df_ori["date"] = pd.to_datetime(df_ori["date"])
df_ori.set_index('date', inplace=True)
df_ts = df_ori.sort_index()
df_test_ts = df_ts.loc[test_start_date:test_end_date]
df_test = df_test_ts.reset_index(drop=True)
df_test

,psp_reference,merchant,card_scheme,year,hour_of_day,minute_of_hour,day_of_year,is_credit,eur_amount,ip_country,issuing_country,device_type,ip_address,email_address,card_number,shopper_interaction,zip_code,card_bin,has_fraudulent_dispute,is_refused_by_adyen
0,70826623779,Merchant A,MasterCard,2021,12,16,335,False,30.34,ZW,IT,Android,cqHg4ONJtOBwqNNSjawwVg,vpCrxKV4Rb4hHYZ_xUQrnA,92TKnNIoPTXgok35txpjyg,Ecommerce,ZB,4916,False,False
1,54770322082,Merchant C,MasterCard,2021,10,44,335,True,37.61,IT,ZW,Linux,I4zf2g3Z4BdSVbOtXDmw7g,YKy04vP5R13SJl5ydrfr3w,iWwHGPT5vcubXqewt6QmFQ,Ecommerce,NaN,4556,False,False
2,76421531318,Merchant E,Other,2021,8,38,335,True,18.87,GR,GR,Linux,M3g-untYinyAQiAIXKGNSw,f-37afIGJJkYJHQXM_HKMg,HDPyxyvo2Lqebm-qusJqOQ,Ecommerce,DFFF,4556,False,False
3,53221980222,Merchant A,Visa,2021,22,33,335,True,31.02,BR,BR,Windows,KT8qTQ6tiACsTKJsyxvPIg,36VSxpSABRjj43cfP6lwjw,gbsYb_MOBHH6FXa_a7c7og,POS,AAD,4571,False,False
4,21921666536,Merchant B,Visa,2021,16,29,335,True,161.40,NL,ZW,Other,wpnDR446pjLAUoNYueB-yg,5MO1Y0cW3sRJ56IbZCwOIQ,s0sj9IsbO7VMzyIsRBBZOA,Ecommerce,2039,4916,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11646,62369458306,Merchant D,Amex,2021,13,27,365,True,112.66,NL,NL,Windows,NUZxGz6wWtW1enn19ghi4g,D5nXKSWwpD40xABXiW5drg,2XeZsqgafyXVZrQNZpC7Kw,Ecommerce,2039,4017,False,False
11647,61422983781,Merchant A,Visa,2021,2,18,365,False,21.38,BR,BR,Windows,-0zsia1FtEIuWYt71pLrKw,NaN,leow_iHIe06J6n5YLGOgyQ,Ecommerce,FFR,4133,False,False
11648,83634178595,Merchant A,MasterCard,2021,11,7,365,True,43.16,NL,NL,iOS,sDicZTCFj6lP8AiV6WFC7g,NJPPoAQt0eoYvDtBIf5tHQ,Gd03xMeOIMabaEfpvqR1wA,Ecommerce,1093,4571,False,False
11649,45741664315,Merchant A,Other,2021,6,35,365,True,16.62,IT,NL,Android,hAKt2EU3vmjrGOT6ueNsdw,3zRwZ-rq1GTbTuPpE9mwXw,h_1om9k6GCO7nv5NeoDgfQ,Ecommerce,NaN,4526,False,False


In [10]:
# append categorical dummies
df_test['no_ip'] = df_test['ip_address'].isnull()
df_test['no_email'] = df_test['email_address'].isnull()
df_test['same_country'] = df_test['ip_country'] == df_test['issuing_country']

# one hot encoding
merchant_dummies = pd.get_dummies(df_test.merchant, prefix='merchant', drop_first=True)
card_scheme_dummies = pd.get_dummies(df_test.card_scheme, prefix='card_scheme', drop_first=True)
ip_country_dummies = pd.get_dummies(df_test.ip_country, prefix='ip_country', drop_first=True)
issuing_country_dummies = pd.get_dummies(df_test.issuing_country, prefix='issuing_country',  drop_first=True)
device_type_dummies = pd.get_dummies(df_test.device_type, prefix='device_type',  drop_first=True)
shopper_interaction_dummies = pd.get_dummies(df_test.shopper_interaction, prefix='shopper_interaction',  drop_first=True)
zip_code_dummies = pd.get_dummies(df_test.zip_code, prefix='zip_code', drop_first=True)
df_test = pd.concat(
    [df_test,merchant_dummies, card_scheme_dummies, ip_country_dummies, issuing_country_dummies, device_type_dummies,
     shopper_interaction_dummies, zip_code_dummies], axis=1)

In [13]:
# append graph features
train_end_date = "2021-11-30"
df_graph_features_lookup_table = pd.read_csv(f"../feature-engineering/graph_features_lookup_table_{train_end_date}.csv")
def append_graph_features(node_type_name, df, graph_feature_lookup_table):
    def append_graph_features_by_row(row):
        node_key = row[node_type_name]
        prefix = node_type_name.split('_')[0]
        try:
            row[f"{prefix}_node_degree"] = graph_feature_lookup_table[graph_feature_lookup_table["Node"] == node_key]["DEGREE"].iloc[0]
            row[f"{prefix}_eigen_centrality"] = graph_feature_lookup_table[graph_feature_lookup_table["Node"] == node_key]["EIGENVECTOR_CENTRALITY"].iloc[0]
            row[f"{prefix}_page_rank"] = graph_feature_lookup_table[graph_feature_lookup_table["Node"] == node_key]["PAGE_RANK"].iloc[0]
        except IndexError as e:
            row[f"{prefix}_node_degree"] = 2
            row[f"{prefix}_eigen_centrality"] = 0.33
            row[f"{prefix}_page_rank"] = 0.57
        return row
    df = df.apply(append_graph_features_by_row, axis=1)
    return df

df_test = append_graph_features("ip_address",df_test, df_graph_features_lookup_table)
df_test = append_graph_features("email_address",df_test, df_graph_features_lookup_table)
df_test = append_graph_features("card_number",df_test, df_graph_features_lookup_table)

In [14]:
# append woe features
df_woe_features = pd.read_csv(f"../feature-engineering/woe_features_{train_end_date}.csv")
def append_woe_features(col_name, df, woe_features):
    def append_woe_features_by_row(row):
        node_key = row[col_name]
        try:
            row[f"{col_name}_woe"] = woe_features[woe_features[col_name] == node_key][f"{col_name}_woe"].iloc[0]
        except IndexError as e:
            row[f"{col_name}_woe"] = 0.0
        return row
    df = df.apply(append_woe_features_by_row, axis=1)
    return df
df_test = append_woe_features("ip_address",df_test,df_woe_features)
df_test = append_woe_features("email_address",df_test,df_woe_features)
df_test = append_woe_features("card_number",df_test,df_woe_features)

In [15]:
df_test.head()

,psp_reference,merchant,card_scheme,year,hour_of_day,minute_of_hour,day_of_year,is_credit,eur_amount,ip_country,...,ip_page_rank,email_node_degree,email_eigen_centrality,email_page_rank,card_node_degree,card_eigen_centrality,card_page_rank,ip_address_woe,email_address_woe,card_number_woe
0,70826623779,Merchant A,MasterCard,2021,12,16,335,False,30.34,ZW,...,0.000004,2,3.184233e-03,0.000004,3,3.198447e-03,0.000007,0.000000,1.392115,1.104433
1,54770322082,Merchant C,MasterCard,2021,10,44,335,True,37.61,IT,...,0.000007,2,-8.414725e-21,0.000007,2,1.576474e-19,0.000007,0.000000,0.000000,0.000000
2,76421531318,Merchant E,Other,2021,8,38,335,True,18.87,GR,...,0.570000,2,3.300000e-01,0.570000,2,3.300000e-01,0.570000,0.000000,0.000000,0.000000
3,53221980222,Merchant A,Visa,2021,22,33,335,True,31.02,BR,...,0.000005,5,3.212985e-03,0.000010,4,3.198770e-03,0.000007,1.392115,0.881290,0.881290
4,21921666536,Merchant B,Visa,2021,16,29,335,True,161.40,NL,...,0.570000,2,3.184169e-03,0.000004,2,3.300000e-01,0.570000,0.000000,0.000000,0.000000


In [16]:
df_test.to_csv("test_dataset_december.csv", index=False)